In [3]:
require 'csv'
puts 'Event Manager Initialized!'

Event Manager Initialized!


In [10]:
contents = CSV.open(
  '../event_attendees.csv',
  headers: true, # TBD - does it mean that this dataset has a header row?
  header_converters: :symbol # what does this mean actually?
)

contents.each do |row|
  name = row[:first_name]
  zipcode = row[:zipcode]
  if zipcode.nil? # otherwise the `length` method cannot be applied to a nil value
    zipcode = '00000' 
  elsif zipcode.length < 5
    zipcode = zipcode.rjust(5, '0') # right-justify a string by padding it with a specific character
  elsif zipcode.length > 5
    zipcode = zipcode[0..4]
  end

  puts "#{name} #{zipcode}"
end

Allison 20010
SArah 20009
Sarah 33703
David 07306
Chris 00000
Aya 90210
Mary Kate 21230
Audrey 95667
Shiyu 96734
Eli 92037
Colin 02703
Megan 43201
Meggie 94611
Laura 00924
Paul 14517
Shannon 03082
Shannon 98122
Nash 98122
Amanda 14841


2126